# B-state energy levels 
Calculating the energy levels and eigenstates of the B-state Hamiltonian

Import the relevant packages and Hamiltonians

In [3]:
import numpy as np
import pickle
import sys
import sympy

sys.path.append('../B-state-classes-and-functions/')

from classes import CoupledBasisState, UncoupledBasisState, State
from functions import matrix_to_states, vector_to_state

with open("B_hamiltonians_symbolic.py",'rb') as f:
    H_dict = pickle.load(f)
    Hff_m = H_dict["Hff"]

    
hamiltonians = {"Hff": Hff_m}

def ni_range(x0, x1, dx=1):
    # sanity check arguments
    if dx==0:
        raise ValueError("invalid parameters: dx==0")
    if x0>x1 and dx>=0:
        raise ValueError("invalid parameters: x0>x1 and dx>=0")
    if x0<x1 and dx<=0:
        raise ValueError("invalid parameters: x0<x1 and dx<=0")
        
    # generate range list
    range_list = []
    x = x0
    while x < x1:
        range_list.append(x)
        x += dx
    return range_list

#Generate quantum numbers
Jmin = 1
Jmax = 3 # max J value in Hamiltonian
#Jmax = 6
I_Tl = 1/2             # I1 in Ramsey's notation
I_F  = 1/2             # I2 in Ramsey's notation


QN = [UncoupledBasisState(J,mJ,I_Tl,m1,I_F,m2z)
      for J  in ni_range(Jmin, Jmax+1)
      for mJ in ni_range(-J,J+1)
      for m1 in ni_range(-I_Tl,I_Tl+1)
      for m2 in ni_range(-I_F,I_F+1)
     ]

In [7]:
variables = [
    *sympy.symbols('Brot Drot H_const'),
    *sympy.symbols('h1_Tl h1_F'),
    sympy.symbols('q'),
    sympy.symbols('c1'),
    sympy.symbols('c_Tl_p')
]

lambdified_hamiltonians = {
    H_name : sympy.lambdify(variables, H_matrix)
    for H_name, H_matrix in hamiltonians.items()
}

In [11]:
#Constants in MHz
Brot = 6687.879e6
Drot = 0.010869e6
H_const = 8.1e-2
h1_Tl = 28789e6
h1_F = 861e6
q = 2.423e6
c1 = -7.83e6
c_Tl_p = 11.17e6

H = {
    H_name : H_fn(
        Brot, Drot, H_const,
        h1_Tl, h1_F,
        q,
        c1,
        c_Tl_p
    )
    for H_name, H_fn in lambdified_hamiltonians.items()
}
H_tot = H["Hff"]

In [12]:
print(np.allclose(H["Hff"], H["Hff"].conj().T))

True


In [13]:
D, V = np.linalg.eigh(H_tot)

In [14]:
states = matrix_to_states(V,QN)

In [15]:
states[2].remove_small_components().transform_to_coupled().print_state()

+0.6391+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 0, I1 = 1/2, I2 = 1/2, Omega = 1>
-0.7686+0.0000j x |J = 1, F1 = 1/2, F = 1, mF = 1, I1 = 1/2, I2 = 1/2, Omega = 1>
